# Iterating on a Notebook

Let's redefine the model training workflow from the previous step in a notebook.

First, we import `union` and the necessary packages:

In [ ]:
import union
import pandas as pd
from sklearn.neural_network import MLPClassifier

Then we define the container image, our tasks, and the workflow:

In [19]:
image = union.ImageSpec(packages=["pandas", "pyarrow", "scikit-learn", "flytekit==1.14.0b6"])

@union.task(container_image=image)
def get_data() -> pd.DataFrame:
    """Get the wine dataset."""
    from sklearn.datasets import load_wine

    print("Getting data")
    data = load_wine(as_frame=True).frame
    return data.assign(target=lambda x: x["target"].where(x["target"] == 0, 1))


@union.task(container_image=image)
def train_model(max_iter: int, hidden_layer_sizes: list[int], data: pd.DataFrame) -> MLPClassifier:
    """Train a model on the wine dataset."""
    print("Training model")
    features = data.drop("target", axis="columns")
    target = data["target"]
    model = MLPClassifier(max_iter=max_iter, hidden_layer_sizes=hidden_layer_sizes)
    return model.fit(features, target)


@union.workflow
def training_workflow(
    max_iter: int = 50,
    hidden_layer_sizes: list[int] = [100, 100],
) -> MLPClassifier:
    """Put all of the steps together into a single workflow."""
    data = get_data()
    return train_model(max_iter, hidden_layer_sizes, data)

## Running the workflow locally

We can run the workflow locally by calling the `training_workflow` as a
Python function, which returns a `MLPClassifier` object:

In [ ]:
training_workflow(max_iter=50)

## Running the workflow remotely

To run the workflow remotely, create a `UnionRemote` object.

In [ ]:
from union.remote import UnionRemote

serverless = UnionRemote()

Then, run the workflow by passing the `training_workflow` function into the
`serverless.execute` method:


In [ ]:
execution = serverless.execute(training_workflow, inputs={"max_iter": 500})
execution

Inspect the execution on the Union dashboard by clicking on the link above 👆.

## Pulling workflow outputs into the notebook

To pull the outputs of a workflow execution into the notebook, we can wait for
the execution to complete and then access the outputs:

In [ ]:
execution.wait(poll_interval=1)
model = execution.outputs['o0']
model

Let's generate some predictions on the training dataset:

In [ ]:
from sklearn.datasets import load_wine

data = load_wine(as_frame=True).frame.sample(10, random_state=123)
features = data.drop("target", axis="columns")
model.predict(features)